In [1]:
import pandas as pd
import numpy as np
import warnings

# Suppress warnings for a cleaner notebook
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("--- Loading ALL Raw CSV Files for v2 Feature Engineering ---")

# Load the core files we know we need
try:
    df_user = pd.read_csv('../data/user_list.csv')
    df_coupon_list = pd.read_csv('../data/coupon_list_train.csv')
    df_visit = pd.read_csv('../data/coupon_visit_train.csv')
    
    # This file contains the ACTUAL purchases
    df_detail = pd.read_csv('../data/coupon_detail_train.csv')

    print(f"Loaded user_list.csv (Shape: {df_user.shape})")
    print(f"Loaded coupon_list_train.csv (Shape: {df_coupon_list.shape})")
    print(f"Loaded coupon_visit_train.csv (Shape: {df_visit.shape})")
    print(f"Loaded coupon_detail_train.csv (Shape: {df_detail.shape})")

except Exception as e:
    print(f"Error loading files: {e}")

--- Loading ALL Raw CSV Files for v2 Feature Engineering ---
Loaded user_list.csv (Shape: (22873, 6))
Loaded coupon_list_train.csv (Shape: (19413, 24))
Loaded coupon_visit_train.csv (Shape: (2833180, 8))
Loaded coupon_detail_train.csv (Shape: (168996, 6))


In [2]:
# df_detail contains the ACTUAL purchases.
# We need to know 'which user purchased which coupon'.

# We only need these 3 columns
df_purchases = df_detail[['USER_ID_hash', 'COUPON_ID_hash', 'ITEM_COUNT']].copy()

# A user could buy the same coupon twice. Let's aggregate.
df_purchases = df_purchases.groupby(['USER_ID_hash', 'COUPON_ID_hash']).sum().reset_index()

# Create our base flag
df_purchases['purchased_this_coupon'] = 1

print("Created 'ground truth' of actual purchases.")
display(df_purchases.head())

Created 'ground truth' of actual purchases.


,USER_ID_hash,COUPON_ID_hash,ITEM_COUNT,purchased_this_coupon
0,0000b53e182165208887ba65c079fc21,38beeadfe3f97e640367eddae4a8c1b5,2,1
1,00035b86e6884589ec8d28fbf2fe7757,25a27d420caa1c46a8d3c0572d27868a,2,1
2,0005b1068d5f2b8f2a7c978fcfe1ca06,4a79cd05ecb2bf8672e1d955f5faa7fa,1,1
3,0005b1068d5f2b8f2a7c978fcfe1ca06,f0f66195d527a5a9509e139ed367b879,1,1
4,000cc06982785a19e2a2fdb40b1c9d59,229ff5cc21c8d26615493be7f3b42841,1,1


In [3]:
# --- Build the Master Dataframe ---

# 1. Starting with the 'visit' log 
df_master = df_visit.copy()
print(f"Starting with 'visit' data: {df_master.shape}")

# 2. Merging the 'user' data (AGE, SEX_ID)
df_master = pd.merge(df_master, df_user, on='USER_ID_hash', how='left')
print(f"After merging 'user' data: {df_master.shape}")

# 3. Merging the 'coupon_list' data (PRICE_RATE, etc.)
df_master = pd.merge(
    df_master, 
    df_coupon_list, 
    left_on='VIEW_COUPON_ID_hash', 
    right_on='COUPON_ID_hash', 
    how='left', 
    suffixes=('_visit', '_coupon')
)
print(f"After merging 'coupon_list' data: {df_master.shape}")

# 4. Merging our base
df_master = pd.merge(
    df_master,
    df_purchases,
    left_on=['USER_ID_hash', 'VIEW_COUPON_ID_hash'],
    right_on=['USER_ID_hash', 'COUPON_ID_hash'],
    how='left'
)
print(f"After merging 'purchases' data: {df_master.shape}")

# 5. Create our FINAL Target Variable
# If 'purchased_this_coupon' is 1, they bought it.
# If it's NaN, they just viewed it but never bought it.
df_master['TARGET_PURCHASE'] = df_master['purchased_this_coupon'].fillna(0).astype(int)

# We will now drop the old, confusing 'PURCHASE_FLG'
df_master = df_master.drop(columns=['PURCHASE_FLG', 'purchased_this_coupon', 'ITEM_COUNT'])

print("\n--- Master Dataframe Created ---")
print("Target variable 'TARGET_PURCHASE' created.")
print(df_master['TARGET_PURCHASE'].value_counts(normalize=True))
display(df_master.head())

Starting with 'visit' data: (2833180, 8)
After merging 'user' data: (2833180, 13)
After merging 'coupon_list' data: (2833180, 37)
After merging 'purchases' data: (2833180, 40)

--- Master Dataframe Created ---
Target variable 'TARGET_PURCHASE' created.
TARGET_PURCHASE
0    0.783663
1    0.216337
Name: proportion, dtype: float64


,I_DATE,PAGE_SERIAL,REFERRER_hash,VIEW_COUPON_ID_hash,USER_ID_hash,SESSION_ID_hash,PURCHASEID_hash,REG_DATE,SEX_ID,AGE,WITHDRAW_DATE,PREF_NAME,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,VALIDPERIOD,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash_x,COUPON_ID_hash_y,TARGET_PURCHASE
0,2012-03-28 14:15:00,7,7d3892e54acb559ae36c459978489330,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,25,NaN,NaN,宅配,宅配,78.0,7200.0,1575.0,2012-03-26 12:00:00,2012-03-30 12:00:00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺,34c48f84026e08355dc3bd19b427f09a,34c48f84026e08355dc3bd19b427f09a,1
1,2012-03-28 14:17:28,9,7d3892e54acb559ae36c459978489330,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,25,NaN,NaN,宅配,宅配,78.0,7200.0,1575.0,2012-03-26 12:00:00,2012-03-30 12:00:00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺,34c48f84026e08355dc3bd19b427f09a,34c48f84026e08355dc3bd19b427f09a,1
2,2012-03-28 14:20:05,16,7d3892e54acb559ae36c459978489330,17c450c3b470c045d35ec22b02daa690,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,25,NaN,NaN,宅配,宅配,66.0,4480.0,1480.0,2012-03-27 12:00:00,2012-03-30 12:00:00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関西,兵庫県,兵庫,17c450c3b470c045d35ec22b02daa690,NaN,0
3,2012-03-28 14:23:16,18,7d3892e54acb559ae36c459978489330,91a15e6a95d09e5e01b50747833b317d,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,25,NaN,NaN,宅配,宅配,80.0,10200.0,1990.0,2012-03-26 12:00:00,2012-03-29 12:00:00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,埼玉県,埼玉,91a15e6a95d09e5e01b50747833b317d,NaN,0
4,2012-03-28 14:26:25,20,7d3892e54acb559ae36c459978489330,96fcbc8f6e45d5a2de1661eb140c6e82,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,25,NaN,NaN,宅配,宅配,85.0,34440.0,4980.0,2012-03-28 12:00:00,2012-04-01 12:00:00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,銀座・新橋・東京・上野,96fcbc8f6e45d5a2de1661eb140c6e82,NaN,0


In [4]:
# Simple Feature Engineering (Row-by-Row)
print("Engineering simple v2 features...")

# 1. Fix dtypes
df_master['I_DATE'] = pd.to_datetime(df_master['I_DATE'])
df_master['VALIDFROM'] = pd.to_datetime(df_master['VALIDFROM'])
df_master['VALIDEND'] = pd.to_datetime(df_master['VALIDEND'])
df_master['REG_DATE'] = pd.to_datetime(df_master['REG_DATE'])

# 2. Time-based features
df_master['visit_day_of_week'] = df_master['I_DATE'].dt.dayofweek
df_master['visit_is_weekend'] = df_master['visit_day_of_week'].isin([5, 6]).astype(int)
df_master['visit_hour'] = df_master['I_DATE'].dt.hour
# ADDED FROM DOCUMENT:
df_master['visit_month'] = df_master['I_DATE'].dt.month

# 3. Coupon-based features
# ADDED FROM DOCUMENT: Calculate raw discount value
df_master['discount_value'] = df_master['CATALOG_PRICE'] - df_master['DISCOUNT_PRICE']

df_master['validity_duration_days'] = (df_master['VALIDEND'] - df_master['VALIDFROM']).dt.days
df_master['validity_duration_days'] = df_master['validity_duration_days'].fillna(0)

# 4. User-based (loyalty) feature
df_master['user_age_days'] = (df_master['I_DATE'] - df_master['REG_DATE']).dt.days
df_master['user_age_days'] = df_master['user_age_days'].fillna(0)

print("Simple features created.")

Engineering simple v2 features...
Simple features created.


In [5]:
# Advanced Feature Engineering (Aggregations)
print("Engineering aggregated behavioral features...")

# --- 1. User-level View/Purchase stats ---
df_user_aggregates = df_master.groupby('USER_ID_hash').agg(
    user_total_views=('VIEW_COUPON_ID_hash', 'count'),
    user_unique_views=('VIEW_COUPON_ID_hash', 'nunique'),
    user_total_purchases=('TARGET_PURCHASE', 'sum')
)
df_user_aggregates['user_conversion_rate'] = df_user_aggregates['user_total_purchases'] / df_user_aggregates['user_total_views']

# --- 2. User-level Preferred Genre ---
df_pref_genre = df_master.groupby('USER_ID_hash')['GENRE_NAME'].agg(
    lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'
).reset_index().rename(columns={'GENRE_NAME': 'user_preferred_genre'})

# --- 3. User-level Repeat View stats ---
df_repeat_views = df_master.groupby(['USER_ID_hash', 'VIEW_COUPON_ID_hash']).size().reset_index(name='views_same_coupon')
df_repeat_user = df_repeat_views.groupby('USER_ID_hash')['views_same_coupon'].max().reset_index(name='user_max_views_same_coupon')

# Avg Time Between Visits ---
# Sort by user and date to calculate time diff
df_sorted = df_master.sort_values(['USER_ID_hash', 'I_DATE'])
df_sorted['time_diff'] = df_sorted.groupby('USER_ID_hash')['I_DATE'].diff().dt.total_seconds()

# Average time gap per user
df_avg_time = df_sorted.groupby('USER_ID_hash')['time_diff'].mean().reset_index(name='avg_time_between_visits')

# --- Merge all aggregates back ---
print("Merging all aggregates back to master...")
df_v2 = pd.merge(df_master, df_user_aggregates, on='USER_ID_hash', how='left')
df_v2 = pd.merge(df_v2, df_pref_genre, on='USER_ID_hash', how='left')
df_v2 = pd.merge(df_v2, df_repeat_user, on='USER_ID_hash', how='left')
# Merge the new time feature
df_v2 = pd.merge(df_v2, df_avg_time, on='USER_ID_hash', how='left')

print(f"Final 'v2' shape before cleaning: {df_v2.shape}")

Engineering aggregated behavioral features...
Merging all aggregates back to master...
Final 'v2' shape before cleaning: (2833180, 52)


In [6]:
# Final Cleaning & Save (CORRECTED)
print("Starting final cleaning...")

# 1. Handle Missing Values (Instead of Dropping Them)
# For numeric features (like time between visits), fill NaNs with -1
# This tells the model "This value is missing/unknown" which is a valid signal.
numeric_cols = df_v2.select_dtypes(include=[np.number]).columns
df_v2[numeric_cols] = df_v2[numeric_cols].fillna(-1)

# For text features, fill with Unknown
object_cols = df_v2.select_dtypes(include=['object']).columns
df_v2[object_cols] = df_v2[object_cols].fillna("Unknown")

# 2. Drop duplicates (This is safe)
df_v2 = df_v2.drop_duplicates()

print(f"Final 'v2' shape after cleaning: {df_v2.shape}")

# 3. Sanity Check: Ensure we didn't lose everyone
if len(df_v2) < 1000:
    print("WARNING: Dataset is suspiciously small. Check logic.")
else:
    print("Dataset size looks healthy.")

# 4. Save the master "v2" features file
output_path = '../data/master_features_v2.csv'
df_v2.to_csv(output_path, index=False)

print(f"\n--- Phase 3 Complete ---")
print(f"Advanced 'v2' features dataset saved to: {output_path}")

Starting final cleaning...
Final 'v2' shape after cleaning: (2833180, 52)
Dataset size looks healthy.

--- Phase 3 Complete ---
Advanced 'v2' features dataset saved to: ../data/master_features_v2.csv
